### 12.010 HOMEWORK 2
GUI to read GPS results, remove trends and plot the data.  
To run this Notebook:
Execute each cell in order until cell labled "Main GUI cell".  Once this cell is executed the figure will be active and can be used.    
The two cells after the Main GUI cell can be used to save the figure generated so far and to output the table of fits in different formats.  Change the logical to variables to control which formats are output.  

Initial look of the figure  
<img src="GUI_Start.png" alt="drawing" style="border: 1px solid #000" width="512"/>  
The options in the figure are:  
- **Overlay**: The default behavior is to overlay plots are different sites and centers are choosen. When clicked, mode is changed to show only one plot with older plots being cleared.
- **Fit/Raw** Radio Box: sets whether a linear trend is removed from the time series.  
- **NEb TextBox**: Sets number of error bars to show: Options:
    - -1 shows line with no error bars (Default)
    -  0 shows points (no line) with no error bar
    -  N shows error bar every Nth point.
- **Off TextBox**: Sets offset bewteen lines when time series are overlaid (Default)
- **UNAV/UNR** Radio Box : set which time series values to diplay
- **Undo**: Removes last time series plot (only works for last one plotted)
- **Plot**: Generated plot given the above settings.  Site TextBox must used with valid 4-char code before plot.
- **Site** TextBox: Enter 4-char codes and use return/enter to submit.  File will be read and time series statistis output. **MUST BE SELECTED FIRST**
  
  
**Example:**  

<img src="GUI_Example.png" alt="drawing" style="border: 1px solid #000" width="512"/>  

Table values for velocity estimates and WRMS Scatters. 
- Vn, Ve and Vu are velocities in North, East and Up (mm/yr)
- σVn, σVe, σVu are sigmas of the velocities assummmg white-noise (not a good assumption)
- WRMS N, WRMS E, WRMS U are weighted root mean scatters of the residuals after linear trend removal (mm)
- χn, χe, χu are the sqrt of $\sqrt{\chi^2/f}$ (NRMS) 
 

|Site|   Vn |   σVn |    Ve |   σVe |    Vu |   σVu |   WRMS N |   χn |   WRMS E |   χe |   WRMS U |   χu |   Num |
|:---|-----:|------:|------:|------:|------:|------:|---------:|-----:|---------:|-----:|---------:|-----:|------:|
|P068-UNAV| 1.63 |  0.01 | -3.70 |  0.01 | -0.00 |  0.02 |     1.21 | 0.59 |     0.91 | 0.57 |     5.75 | 0.79 |  5321 |
|P069-UNAV| 1.03 |  0.01 | -3.31 |  0.00 | -0.27 |  0.02 |     1.37 | 0.68 |     1.12 | 0.71 |     5.82 | 0.81 |  5495 |
|P072-UNAV| 0.84 |  0.01 | -3.11 |  0.00 | -0.54 |  0.02 |     1.57 | 0.78 |     1.36 | 0.86 |     6.00 | 0.84 |  5259 |
|P073-UNAV| 0.48 |  0.01 | -3.02 |  0.00 | -0.82 |  0.02 |     1.18 | 0.59 |     0.78 | 0.50 |     5.51 | 0.78 |  5500 |


In [1]:
# Load up all the modules we need
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from datetime import datetime
from matplotlib.widgets import TextBox, RadioButtons, Button
from tabulate import tabulate


from os.path import exists
#
# Magic to start new plot figure
%matplotlib 

Using matplotlib backend: MacOSX


In [2]:
# Function to read UNAVCO Data
def Read_UNAV(site):
    '''Function to read data from UNAVCO and return numpy array and datetime arrays.
    Input:
       sites - 4-character site name
    Output
       times   - date-times dates thar can be used in plotting
       tseries - time series with days for 2000/1/1, dn,sn, de,se, du, su'''
    
    # Create the file name and URL
    filen = site.upper()+".cwu.nam14.csv"
    whurl="https://data.unavco.org/archive/gnss/products/position/"+site.upper()+"/"+filen
    
    #PBO Station Position Time Series.
    #Format Version, 1.2.0
    #Reference Frame, NAM14
    #4-character ID, P177
    #Station name, CoDeTierraCN2008
    #Begin Date, 2008-05-09
    #End Date, 2021-10-06
    #Release Date, 2021-10-07
    #Source file, P177.cwu.nam14.pos
    #Offset from source file, 166.01 mm North, -139.45 mm East, -3.62 mm Vertical
    #Reference position, 37.5281683684 North Latitude, -122.4950535711 East Longitude, 71.79172 meters elevation
    #Date, North (mm), East (mm), Vertical (mm), North Std. Deviation (mm), East Std. Deviation (mm), Vertical Std. Deviation (mm), Quality,
    #2008-05-09,0.00, 0.00, 0.00, 2.08, 1.66, 7.84, repro,
    #2008-05-10,0.38, 0.83, 2.66, 2.13, 1.7, 7.99, repro,
    #...
    # print(filen, whurl)
    if exists(filen):
        print('Reading from file',filen)
        df = pd.read_csv(filen,delimiter=',')
    else:
        print('Downloading from ',whurl)
        df = pd.read_csv(whurl,delimiter=',',header=11)
        # Write csv local copy for later use
        #print('Creating',filen)
        #df.to_cvs(filen)
        df.to_csv(filen,index=False)


    ninit = len(df)
    # Remove data with NEU sigmas > 10, 10 and 30 mm.
    # Apply sequencially because 'and' does not seem to work (latter use values
    # extracted from GUI box)
    df=df[df.iloc[:,4]<10];
    df=df[df.iloc[:,5]<10];
    df=df[df.iloc[:,6]<30];
    print("Number after >10,10,30 NEU sigma removal ",len(df)," Read",ninit)
    npdat =df.to_numpy()
    # Extract time,and data and sigma
    t=list(npdat[:,0]);nd=list(npdat[:,1]); ed=list(npdat[:,2]); ud=list(npdat[:,3]); 
    ns=list(npdat[:,4]) ; es=list(npdat[:,5]) ; us=list(npdat[:,6])
    # Now convert the time into a datetime type (to) and days from 2000/01/01
    # as float array (td).
    # We can ise to in plotting and td for fitting to data
    n = 0
    to = []
    td = np.zeros(len(t))
    for v in t:
        to = np.append(to,datetime.strptime(v, '%Y-%m-%d'))
        dt = to[n] - datetime(2000, 1, 1)  # Time difference from 2000/1/1
        td[n] = dt.total_seconds()/86400.  # Days from 2000/1/1
        n += 1
        
    # Now construct data array
    tseries = np.array([td,nd,ns,ed,es,ud,us])
    times = to
    return times, tseries


print('ReadUNAV Defined')        

ReadUNAV Defined


In [3]:
# Function to read UNAVCO Data
# from datetime import datetime
def Read_UNR(site):
    '''Function to read data from UNR and return numpy array and datetime arrays.
    Input:
       sites - 4-character site name
    Output
       times   - date-times dates thar can be used in plotting
       tseries - time series with days for 2000/1/1, dn,sn, de,se, du, su'''
    
    # Create the file name and URL
    filen = site.upper()+".NA.tenv3"
    whurl="http://geodesy.unr.edu/gps_timeseries/tenv3/plates/NA/"+filen
    #      http://geodesy.unr.edu/gps_timeseries/tenv3/plates/NA/P068.NA.tenv3
    # http://geodesy.unr.edu/gps_timeseries/tenv3/plates/NA/FOXG.NA.tenv3 1 header
    # site YYMMMDD yyyy.yyyy __MJD week d reflon _e0(m) __east(m) ____n0(m) _north(m) u0(m) ____up(m) _ant(m) sig_e(m) sig_n(m) sig_u(m) __corr_en __corr_eu __corr_nu
    # FOXG 15APR02 2015.2498 57114 1838 4 -118.2  -3709  0.017152   3845073  0.746556   688  0.822390  0.0083 0.000668 0.000799 0.002751  0.059699 -0.077153 -0.102726
    # ...
    #
    #print(filen, whurl)
    if exists(filen):
        print('Reading from file',filen)
        df = pd.read_csv(filen,delimiter=',')
    else:
        print('Downloading from ',whurl)
        df = pd.read_csv(whurl,delimiter=r"\s+",header=1)
        # Write csv local copy for later use
        #print('Creating',filen)
        df.to_csv(filen,index=False)

    ninit = len(df)
    # Remove data with NEU sigmas > 10, 10 and 30 mm.
    # Apply sequencially because 'and' does not seem to work
    df=df[df.iloc[:,14]<10e-3];
    df=df[df.iloc[:,15]<10e-3];
    df=df[df.iloc[:,16]<30e-3];
    print("Number after >10,10,30 NEU sigma removal ",len(df),'Read',nint)
    npdat =df.to_numpy()
    # Extract time 
    t=list(npdat[:,1]);
    nd=list((npdat[:,10]-npdat[0,10])*1000); # remove first value so starts at zero
    # the same a UNAV. (Problem if times of first data point are different)
    ed=list((npdat[:,8]-npdat[0,8])*1000); 
    ud=list((npdat[:,12]-npdat[0,12])*1000); 
    ns=list(npdat[:,15]*1000) ; es=list(npdat[:,14]*1000) ; us=list(npdat[:,16]*1000)
    # Now convert the time into a datetime type (to) and days from 2000/01/01
    # as float array (td).
    # We can ise to in plotting and td for fitting to data
    n = 0
    to = []
    td = np.zeros(len(t))
    for v in t:
        to = np.append(to,datetime.strptime(v, '%y%b%d'))
        dt = to[n] - datetime(2000, 1, 1,0,0)  # Time difference from 2000/1/1
        td[n] = dt.total_seconds()/86400.  # Days from 2000/1/1
        n += 1
        
    # Now construct data array
    tseries = np.array([td,nd,ns,ed,es,ud,us])
    times = to
    return times, tseries

print('ReadUNR Defined')        

ReadUNR Defined


In [4]:
def FitTS(xdata,ydata,sig):
    '''Fit times series.  Linear trend currently
    data is xdata time in days, ydata (NEU), sig 
    '''
    yr = xdata/365.25; # Years Since 2000/1/1
    wgh = 1./sig**2
    ndata = int(xdata.size)  # number of data points
    A = np.transpose([np.ones(ndata),yr])
    NormEQ = np.matmul(np.transpose(A)*wgh,A)
    Bvec = np.matmul(np.transpose(A)*wgh,np.transpose(ydata))
    MCov = np.linalg.inv(NormEQ)
    MEst = np.matmul(MCov,Bvec)
    Res = ydata-np.matmul(A,MEst)
    chi = np.sqrt(np.dot(np.transpose(Res),Res*wgh)/(ndata-2))
    wrms = np.sqrt(ndata/np.sum(wgh))*chi
    pfe = np.flip(MEst) # Set reverse order like polyfit
    #print("Est  : a=%5.3f, b=%5.3f" % tuple(MEst) )
    #print("Sig  : a=%5.3f, b=%5.3f" % tuple(np.sqrt(np.diag(MCov))) )
    return pfe, [MEst[1],np.sqrt(MCov[1,1])], [wrms,chi,ndata]

print('FitTS Defined')

FitTS Defined


In [5]:
#if (systype=='darwin' or systype == 'linux2'):
#    !echo "axes.titlesize : 24" > my.style
#    !echo "axes.labelsize : 20" >> my.style
#    !echo "lines.linewidth : 3" >> my.style
#    !echo "lines.markersize : 10" >> my.style
#    !echo "xtick.labelsize : 16" >> my.style
#    !echo "ytick.labelsize : 16" >> my.style
#    print('Created my.style')
plt.rcParams['lines.linewidth'] = 1
plt.rcParams['axes.labelsize'] = 8
plt.rcParams['xtick.labelsize'] = 8
plt.rcParams['ytick.labelsize'] = 8
plt.rcParams['lines.markersize'] = 2

In [6]:
# Main plot loop and GUI setup.
def PlotGUIA(tpt,dpt,fig):
    ''' Plot to axes already defined in main notebook'''
    global ax1, ax2, ax3, axbox, sitename, Center, detrend, new, clf \
       , NEB, dYoff, Yoff
    #print('Plot STATUS ',sitename, Center, detrend, NEB, dYoff, Yoff)

    # Fit a linear line to the North data
    # pn = np.polyfit(dpt[0,:]-dpt[0,0],dpt[1,:],1)
    # Polyfit with sigmas (same result as FitTS but FitTS can be 
    #   extended with addition parameters)
    # pn, cov = np.polyfit(dpt[0,:]/365.25,dpt[1,:],1,cov='unscaled',w=1./dpt[2,:])
    pn, Vel, Stat = FitTS(dpt[0,:],dpt[1,:],dpt[2,:])
    # Initialize for data frame result return
    VelNEU = Vel ; StatNEU = Stat[0:2]; # Just save WRMS and χ
    #print('Fit ',pn,pfe)
    # Add code to see if residual to be plotted
    nr = dpt[1,:] + Yoff
    if detrend == 1:
        nr = dpt[1,:] - np.polyval(pn,dpt[0,:]/365.25)+Yoff
    if new == 1:    
        ax1 = fig.add_subplot(311)
        
    if clf:
        ax1.clear() # Clears 
        ax2.clear()
        ax3.clear()
 
    legnd = sitename+'-'+Center
    #print('Legend',legnd)
    line = []
    if NEB > 0:
        l1 = ax1.errorbar(tpt,nr,dpt[2,:],marker='.', 
                 linestyle=' ',capsize=4,errorevery=NEB,label=legnd)
    elif NEB == 0:
        l1 = ax1.plot(tpt,nr,marker='.',linestyle=' ',label=legnd)
    else:
        l1 = ax1.plot(tpt,nr,label=legnd)


    line += l1 
    # Fit a linear line to the East data
    #pe = np.polyfit(dpt[0,:]/365.25,dpt[3,:],1)
    pe, Vel, Stat = FitTS(dpt[0,:],dpt[3,:],dpt[4,:])
    VelNEU += Vel ; StatNEU += Stat[0:2]; # Just save WRMS and χ

    er = dpt[3,:] + Yoff
    #print('Yoff',Yoff,er[0],dpt[3,0])
    if detrend == 1:
        er = dpt[3,:] - np.polyval(pe,dpt[0,:]/365.25) + Yoff
    if new == 1:    
        ax2 = fig.add_subplot(312)
    if NEB > 0:
        l2 = ax2.errorbar(tpt,er,dpt[4,:],marker='.', 
             linestyle=' ',capsize=4,errorevery=NEB)
    elif NEB==0:
        l2 = ax2.plot(tpt,er,marker='.',linestyle=' ')
    else:
        l2 = ax2.plot(tpt,er)


    line += l2 
    #ax2.plot(tpt,er,'+')

    # pu = np.polyfit(dpt[0,:]-dpt[0,0],dpt[5,:],1)
    pu, Vel, Stat = FitTS(dpt[0,:],dpt[5,:],dpt[6,:])
    VelNEU += Vel ; StatNEU += Stat; # Append WRMS, χ and num
    ur = dpt[5,:] + 3*Yoff
    if detrend == 1:
        ur = dpt[5,:] - np.polyval(pu,dpt[0,:]/365.25)+3*Yoff
    if new == 1:    
        ax3 = fig.add_subplot(313)
    if NEB > 0:
        l3 = ax3.errorbar(tpt,ur,dpt[6,:],marker='.', 
                 linestyle=' ',capsize=4,errorevery=NEB)
    elif NEB == 0:
        l3 = ax3.plot(tpt,ur,marker='.',linestyle=' ')
    else:
        l3 = ax3.plot(tpt,ur)
        
    line += l3

    # Link axes
    ax1.get_shared_x_axes().join(ax1, ax2, ax3)
    #ax1.set_xticklabels([])
    #ax2.set_xticklabels([])
    # Now reset positions of xaxes so that touch
    ax1.set_position((0.20, 0.66,0.68,0.29))
    ax2.set_position((0.20, 0.37,0.68,0.29))
    ax3.set_position((0.20, 0.08,0.68,0.29))
    
    # Set y-axis labels
    ax1.set_ylabel('ΔNorth (mm)')
    ax2.set_ylabel('ΔEast (mm)')
    ax3.set_ylabel('ΔHeight (mm)')
    ax3.set_xlabel('Date')

    # Change tick formats
    ax1.tick_params(direction='in', length=6, width=1,
            right=True,top=True,labelsize=8)
    ax2.tick_params(direction='in', length=6, width=1,
            right=True,top=True,labelsize=8)
    ax3.tick_params(direction='in', length=6, width=1,
            right=True,top=True,labelsize=8)

    #ax1.legend()
    #ax1.legend(bbox_to_anchor=(1.01, 0.99))
    ax1.legend(bbox_to_anchor=(1.00, 1), loc='upper left', borderaxespad=0.)


    # Set up site name
    #axbox = fig.add_axes([0.05, 0.04, 0.15, 0.03])
    #text_box = TextBox(axbox, "Site")
    #text_box.set_val("P068") 
    #site = text_box.on_submit(NewSite)
    #print('Processing site',site)

    #ax3.xaxis.set_major_locator(plt.MaxNLocator(4))
    #plt.show(); , index=legnd
    new = 0;
    #print('VelNEU',VelNEU)
    #print('StatNEU',StatNEU)
    
    # New entry for data frame
    dfa = pd.DataFrame([VelNEU+StatNEU], \
         columns=['Vn','σVn','Ve','σVe','Vu','σVu','WRMS N','χn','WRMS E','χe','WRMS U','χu','Num'],index=[legnd])

    display(dfa.style.format('{:.2f}'))
    #display(dfa.style.format(precision=2))  # Does not seem to work


    return line, dfa


print('PlotGIA Defined')

PlotGIA Defined


### Main GUI cell
Once the cell below is executed, figure will be active.  Basic procedure is to enter the 4-char code of as site in the lower TexBox, press enter to submit the name request and then plot the time series.

In [7]:
######################################
# MAIN GUI Cell: Once Run figure will be active
######################################

plt.rcParams['figure.figsize'] = [11, 8.5]
fig = plt.figure()
# Needed to stop deprecated warning message on text boxes
fig.canvas.mpl_disconnect(fig.canvas.manager.key_press_handler_id)
sitename = '   '
Center = 'UNAV'
detrend = 1 # Default is not to detrend data.
NEB = -1 # Number of error bars: -1 Draw line, 0 points no errorbar, >0 frequency of error bars
dYoff = 0.0 # Amount to offset each by (3 times for H)
Yoff = 0.00 # Current Y-off (reset when clear used)
new = 1 
clf = False
line = []
df = pd.DataFrame(columns=['Vn','σVn','Ve','σVe','Vu','σVu','WRMS N','χn','WRMS E','χe','WRMS U','χu','Num'])

axcolor = 'lightgoldenrodyellow'

axbox = fig.add_axes([0.05, 0.04, 0.06, 0.03])
text_box = TextBox(axbox, "Site")
text_box.set_val("") 

# Create 2nd  `matplotlib.widgets.Button` to see what happens.
overax = plt.axes([0.05, 0.91, 0.06, 0.04])
buttclr = Button(overax, 'OverLay', color=axcolor, hovercolor='0.9')

# Create a `matplotlib.widgets.Button` to reset the sliders to initial values.
detax = plt.axes([0.05, 0.84, 0.06, 0.06], facecolor=axcolor)
radiode = RadioButtons(detax, ('Fit','Raw'))

nebax = plt.axes([0.05, 0.78, 0.06, 0.03], facecolor=axcolor)
neb_box = TextBox(nebax, "NEb")
neb_box.set_val(str(NEB)) 

offax = plt.axes([0.05, 0.74, 0.06, 0.03], facecolor=axcolor)
off_box = TextBox(offax, "Off")
off_box.set_val(str(dYoff)) 


rax = plt.axes([0.05, 0.16, 0.06, 0.06], facecolor=axcolor)
radiouu = RadioButtons(rax, ('UNAV', 'UNR'))

undoax = plt.axes([0.05, 0.12, 0.06, 0.03])
buttund = Button(undoax, 'Undo', color=axcolor, hovercolor='0.9')

plotax = plt.axes([0.05, 0.08, 0.06, 0.03])
buttplt = Button(plotax, 'Plot', color='sandybrown', hovercolor='0.9')

def Plot(event):
    global tpt,dpt, line, Yoff, dYoff,df
    '''
    Plot the times series
    '''
    Yoff += dYoff
    line, dfa = PlotGUIA(tpt,dpt,fig)
    df = df.append(dfa)
    #display(df.style.format('{:.2f}'))
    plt.draw()

def CenSelect(label):
    global Center
    hzdict = {'UNAV': 'UNAV', 'UNR': 'UNR'}
    # Get the name of the center (Defualt is UNAV)
    Center = hzdict[label]
    print('Center Choice',Center)
    #l.set_ydata(ydata)
    #plt.draw()

def Detrend(label):
    global detrend
    detdict = {'Raw': 0, 'Detrend': 1}
    # Get the name of the center (Defualt is UNAV)
    detrend = detdict[label]
    print('Detrend',detrend)
    
def OverLay(event):
    global clf, Yoff, ax1, ax2, ax3, df
    # Add clear() so user sees immediate change
    if clf:
        clf = False
    else:
        clf = True
        
    Yoff = 0.
    if clf:
        ax1.clear() # Clears 
        ax2.clear()
        ax3.clear()
        
    print('OverLay Figure Status', clf)
    display(df.style.format({'Vn': "{:.2f}",'σVn':"{:.2f}",'Ve':"{:.2f}",'σVe':"{:.2f}",'Vu':"{:.2f}",'σVu':"{:.2f}", \
                         'WRMS N':"{:.2f}",'χn':"{:.2f}",'WRMS E':"{:.2f}",'χe':"{:.2f}",'WRMS U':"{:.2f}",'χu':"{:.2f}"}))
    # Now clear for next btach of plots
    df = pd.DataFrame(columns=['Vn','σVn','Ve','σVe','Vu','σVu','WRMS N','χn','WRMS E','χe','WRMS U','χu','Num'])
    
def SetNEB(string):
    global sitename, Center, NEB
    
    NEB = int(string)
    #print('NEB',NEB)
    
def SetOff(string):
    global dYoff, Yoff
    dYoff = int(string)
    #print('dYoff', dYoff, Yoff)
    
    
# Plot new data
def NewSite(site):
    global sitename, Center, tpt, dpt
    #print('NewPlot name Old',sitename, 'Center',Center)
    oldsite = sitename
    sitename = site
    #print('NewPlot name New',sitename)
    if (Center == 'UNAV') :
        tpt, dpt = Read_UNAV(sitename)
    elif ( Center == 'UNR'):
        tpt, dpt = Read_UNR(sitename)

    
def rmline(event):
    ''' Function to remove lines from plots '''
    global line
    print('rmline',len(line),event)
    if len(line)==3:
        # Simple line plot; remove each one
        for k in range(3):
            l = line[k] 
            l.remove()
    elif len(line)==9 :
        for k in range(3):
            l = line[k*3]
            l.remove()
            for l in line[k*3+1]:
                l.remove()
            for l in line[k*3+2]:
                l.remove()
            

radiode.on_clicked(Detrend)
buttclr.on_clicked(OverLay)
buttplt.on_clicked(Plot)
buttund.on_clicked(rmline)
radiouu.on_clicked(CenSelect)

neb_box.on_submit(SetNEB)
off_box.on_submit(SetOff)

text_box.on_submit(NewSite);
print('GUI Started, Select Site <cr>')


GUI Started, Select Site <cr>
Reading from file P068.cwu.nam14.csv
Number after >10,10,30 NEU sigma removal  5321  Read 5322
Reading from file P068.cwu.nam14.csv
Number after >10,10,30 NEU sigma removal  5321  Read 5322


,Vn,σVn,Ve,σVe,Vu,σVu,WRMS N,χn,WRMS E,χe,WRMS U,χu,Num
P068-UNAV,1.63,0.01,-3.70,0.01,-0.00,0.02,1.21,0.59,0.91,0.57,5.75,0.79,5321.00


Reading from file P069.cwu.nam14.csv
Number after >10,10,30 NEU sigma removal  5495  Read 5496
Reading from file P069.cwu.nam14.csv
Number after >10,10,30 NEU sigma removal  5495  Read 5496


,Vn,σVn,Ve,σVe,Vu,σVu,WRMS N,χn,WRMS E,χe,WRMS U,χu,Num
P069-UNAV,1.03,0.01,-3.31,0.00,-0.27,0.02,1.37,0.68,1.12,0.71,5.82,0.81,5495.00


Reading from file P073.cwu.nam14.csv
Number after >10,10,30 NEU sigma removal  5500  Read 5503
Reading from file P073.cwu.nam14.csv
Number after >10,10,30 NEU sigma removal  5500  Read 5503


,Vn,σVn,Ve,σVe,Vu,σVu,WRMS N,χn,WRMS E,χe,WRMS U,χu,Num
P073-UNAV,0.48,0.01,-3.02,0.00,-0.82,0.02,1.18,0.59,0.78,0.50,5.51,0.78,5500.00


In [ ]:
# Save figure cell: Change the name of the file to print current figure 
# to a file.  (TextBox could be added to do this)
fig.savefig('GUI_Example.png')

In [8]:
# Use this cell to print fits to data so far
# Set Output types wanted
MD = True
Word = True
Latex = True
print('Result so far')
display(df.style.format({'Vn': "{:.2f}",'σVn':"{:.2f}",'Ve':"{:.2f}",'σVe':"{:.2f}",'Vu':"{:.2f}",'σVu':"{:.2f}", \
                         'WRMS N':"{:.2f}",'χn':"{:.2f}",'WRMS E':"{:.2f}",'χe':"{:.2f}",'WRMS U':"{:.2f}",'χu':"{:.2f}"}))


headers = ['Site','Vn','σVn','Ve','σVe','Vu','σVu','WRMS N','χn','WRMS E','χe','WRMS U','χu','Num']
table = df.to_numpy()
index = np.array(list(df.index.values))
fulltable = np.c_[index , table]
if MD :
    print('\nPIPE: For pandas.to_markdown')
    print(tabulate(fulltable, headers, tablefmt="pipe",floatfmt=(".2f", ".2f",".2f",".2f",".2f", ".2f",".2f",".2f",".2f", ".2f",".2f",".2f")))
    
if Word:
    print('\nTab separated for Word')
    print(tabulate(fulltable,headers,tablefmt='tsv',floatfmt=(".2f", ".2f",".2f",".2f",".2f", ".2f",".2f",".2f",".2f", ".2f",".2f",".2f",".2f")))
    
if Latex:
    print('\nLaTex Table (unicode σ and χ may need $\sigma$ and $\chi$)')
    print(tabulate(fulltable,headers,tablefmt='latex',floatfmt=(".2f", ".2f",".2f",".2f",".2f", ".2f",".2f",".2f",".2f", ".2f",".2f",".2f",".2f")))
    

Result so far


,Vn,σVn,Ve,σVe,Vu,σVu,WRMS N,χn,WRMS E,χe,WRMS U,χu,Num
P068-UNAV,1.63,0.01,-3.70,0.01,-0.00,0.02,1.21,0.59,0.91,0.57,5.75,0.79,5321
P069-UNAV,1.03,0.01,-3.31,0.00,-0.27,0.02,1.37,0.68,1.12,0.71,5.82,0.81,5495
P073-UNAV,0.48,0.01,-3.02,0.00,-0.82,0.02,1.18,0.59,0.78,0.50,5.51,0.78,5500



PIPE: For pandas.to_markdown
| Site      |   Vn |   σVn |    Ve |   σVe |    Vu |   σVu |   WRMS N |   χn |   WRMS E |   χe |   WRMS U |       χu |   Num |
|:----------|-----:|------:|------:|------:|------:|------:|---------:|-----:|---------:|-----:|---------:|---------:|------:|
| P068-UNAV | 1.63 |  0.01 | -3.70 |  0.01 | -0.00 |  0.02 |     1.21 | 0.59 |     0.91 | 0.57 |     5.75 | 0.794322 |  5321 |
| P069-UNAV | 1.03 |  0.01 | -3.31 |  0.00 | -0.27 |  0.02 |     1.37 | 0.68 |     1.12 | 0.71 |     5.82 | 0.812764 |  5495 |
| P073-UNAV | 0.48 |  0.01 | -3.02 |  0.00 | -0.82 |  0.02 |     1.18 | 0.59 |     0.78 | 0.50 |     5.51 | 0.782601 |  5500 |

Tab separated for Word
Site     	  Vn	  σVn	   Ve	  σVe	   Vu	  σVu	  WRMS N	  χn	  WRMS E	  χe	  WRMS U	  χu	  Num
P068-UNAV	1.63	 0.01	-3.70	 0.01	-0.00	 0.02	    1.21	0.59	    0.91	0.57	    5.75	0.79	 5321
P069-UNAV	1.03	 0.01	-3.31	 0.00	-0.27	 0.02	    1.37	0.68	    1.12	0.71	    5.82	0.81	 5495
P073-UNAV	0.48	 0.01	-3.02	 0.00